In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import os
from glob import glob
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import preprocess_input

# Feature Extractor

In [10]:
# Load VGG feature extractor
model_vgg16 = keras.applications.vgg16.VGG16(weights=None, input_shape=(256, 256, 3), pooling='avg', include_top=True)


# Dataset

In [11]:
# Data Accessing

# /scratch/ideeps/mateus.miranda/lucai_proj/data/dataset_cerradov3_NIR+G+B_splited_50k
# /Users/mateus.miranda/INPE-CAP/mestrado/LUCIai project/algorithm/cerraNet_v3/data/dataset_cerradov3_NIR+G+B_splited_50k

d_train = '/Users/mateus.miranda/INPE-CAP/mestrado/LUCIai project/algorithm/cerraNet_v3/data/dataset_cerradov3_NIR+G+B_splited_50k/train'
d_test = '/Users/mateus.miranda/INPE-CAP/mestrado/LUCIai project/algorithm/cerraNet_v3/data/dataset_cerradov3_NIR+G+B_splited_50k/test'



In [12]:
# Train
x_train2 = []
x_train = list()
y_train = []
y_label = []
label_names_train = []

for i in glob(os.path.join(os.getcwd(), d_train+'/**/*.tif')):

    # Load x:

    load_img = tf.keras.utils.load_img(i, target_size=(256, 256))
    img = tf.keras.utils.img_to_array(load_img)
    img_x = np.expand_dims(img, axis=0)
    x_train2.append(img_x)
    img_p = preprocess_input(img_x)
    feature = model_vgg16.predict(img_p)
    x_train.append(feature)

    # Load y:

    nameDir = os.path.dirname(i)
    nameFile = os.path.basename(i)
    address = nameDir + '/' + nameFile
    y_label.append(nameDir.split('/')[11]) # 11 8
    label_names_train, y_train = np.unique(y_label, return_inverse=True)

2022-05-15 19:30:48.473515: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [13]:
print(y_train)
print(np.shape(x_train))

[4 4 4 ... 0 0 0]
(40000, 1, 1000)


In [14]:
nsamples, nx, ny = np.shape(x_train)
d2_train_dataset = np.reshape(x_train,(nsamples,nx*ny))

In [15]:
print(d2_train_dataset.shape)

(40000, 1000)


In [16]:
x_test = []
x_test2 = []
y_test = []
y_label_test = []
label_names_test = []

for i in glob(os.path.join(os.getcwd(), d_test+'/**/*.tif')):

    # Load x:

    load_img = tf.keras.utils.load_img(i, target_size=(256, 256))
    img = tf.keras.utils.img_to_array(load_img)
    img_x = np.expand_dims(img, axis=0)
    x_test2.append(img_x)
    img_p = preprocess_input(img_x)
    feature = model_vgg16.predict(img_p)
    x_test.append(feature)
    x_test2.append(img_x)

    # Load y:

    nameDir = os.path.dirname(i)
    nameFile = os.path.basename(i)
    address = nameDir + '/' + nameFile
    y_label_test.append(nameDir.split('/')[11])
    label_names_test, y_test = np.unique(y_label_test, return_inverse=True)

print(y_test)

[4 4 4 ... 0 0 0]


# SUPPORT VECTOR MACHINE

In [17]:
model_svm = svm.SVC().fit(d2_train_dataset, y_train)

In [18]:
nsamples, nx, ny = np.shape(x_test)
d2_test_dataset = np.reshape(x_test,(nsamples,nx*ny))
print(d2_test_dataset.shape)

(5000, 1000)


In [19]:
yp_vgg16 = model_svm.predict(d2_test_dataset)
accuracy_svm = accuracy_score(y_test, yp_vgg16)
f1score_svm = f1_score(y_test, yp_vgg16, average='micro')

In [20]:
print(accuracy_svm)
print(f1score_svm)

0.8548
0.8548000000000001


In [27]:

# CHECKING FARMING CLASS
fr = []   # farming
i = 0     # counter
j = 0     # counter
k = 0     # counter
m = 0     # counter
n = 0     # counter


path = '../../data/dataset_cerradov3_NIR+G+B_splited_50k/test/farming/*.tif'
for img in glob(os.path.join(os.getcwd(), path)):
    # Read the images
    ia = tf.keras.utils.load_img(img, target_size=(256, 256))

    # Transform them in array
    ia = tf.keras.utils.img_to_array(ia)
    ia = np.expand_dims(ia, axis=0)
    img_p = preprocess_input(ia)
    feature = model_vgg16.predict(img_p)

    # Get image information
    path = os.path.dirname(img)
    file_name = os.path.basename(img)
    origin_address = path + '/' + file_name

    # Count image
    print('Reading Image:', file_name, '...')

    # Make predictions
    forecast = model_svm.predict(feature)

    # Image Discriminator:
    if forecast == 0:
        i = i+1
        print('farming')

    if forecast == 1:
        j = j+1
        print('forest_formation')

    if forecast == 2:
        k = k+1
        print('non_forest_area')

    if forecast == 3:
        m = m+1
        print('savanna_formation')

    if forecast == 4:
        n = n+1
        print('water')

fr.append(i)
fr.append(j)
fr.append(k)
fr.append(m)
fr.append(n)

# CHECKING FOREST_FORMATION CLASS
ff = []   # forest_formation
i = 0     # counter
j = 0     # counter
k = 0     # counter
m = 0     # counter
n = 0     # counter

path = '../../data/dataset_cerradov3_NIR+G+B_splited_50k/test/forest_formation/*.tif'
for img in glob(os.path.join(os.getcwd(), path)):
    # Read the images
    ia = tf.keras.utils.load_img(img, target_size=(256, 256))

    # Transform them in array
    ia = tf.keras.utils.img_to_array(ia)
    ia = np.expand_dims(ia, axis=0)

    img_p = preprocess_input(ia)
    feature = model_vgg16.predict(img_p)

    # Get image information
    path = os.path.dirname(img)
    file_name = os.path.basename(img)
    origin_address = path + '/' + file_name

    # Count image
    print('Reading Image:', file_name, '...')

    # Make predictions
    forecast = model_svm.predict(feature)

    # Image Discriminator:
    if forecast == 0:
        i = i+1
        print('farming')

    if forecast == 1:
        j = j+1
        print('forest_formation')

    if forecast == 2:
        k = k+1
        print('non_forest_area')

    if forecast == 3:
        m = m+1
        print('savanna_formation')

    if forecast == 4:
        n = n+1
        print('water')

ff.append(i)
ff.append(j)
ff.append(k)
ff.append(m)
ff.append(n)

# CHECKING NON_FOREST_AREA CLASS
nfa = []  # non_forest_area
i = 0     # counter
j = 0     # counter
k = 0     # counter
m = 0     # counter
n = 0     # counter

path = '../../data/dataset_cerradov3_NIR+G+B_splited_50k/test/non_forest_area/*.tif'
for img in glob(os.path.join(os.getcwd(), path)):
    # Read the images
    ia = tf.keras.utils.load_img(img, target_size=(256, 256))

    # Transform them in array
    ia = tf.keras.utils.img_to_array(ia)
    ia = np.expand_dims(ia, axis=0)

    img_p = preprocess_input(ia)
    feature = model_vgg16.predict(img_p)

    # Get image information
    path = os.path.dirname(img)
    file_name = os.path.basename(img)
    origin_address = path + '/' + file_name

    # Count image
    print('Reading Image:', file_name, '...')

    # Make predictions
    forecast = model_svm.predict(feature)

    # Image Discriminator:
    if forecast == 0:
        i = i+1
        print('farming')

    if forecast == 1:
        j = j+1
        print('forest_formation')

    if forecast == 2:
        k = k+1
        print('non_forest_area')

    if forecast == 3:
        m = m+1
        print('savanna_formation')

    if forecast == 4:
        n = n+1
        print('water')

nfa.append(i)
nfa.append(j)
nfa.append(k)
nfa.append(m)
nfa.append(n)


# CHECKING SAVANNA_FORMATION CLASS
sf = []   # savanna_formation
i = 0     # counter
j = 0     # counter
k = 0     # counter
m = 0     # counter
n = 0     # counter

path = '../../data/dataset_cerradov3_NIR+G+B_splited_50k/test/savanna_formation/*.tif'
for img in glob(os.path.join(os.getcwd(), path)):
    # Read the images
    ia = tf.keras.utils.load_img(img, target_size=(256, 256))

    # Transform them in array
    ia = tf.keras.utils.img_to_array(ia)
    ia = np.expand_dims(ia, axis=0)

    img_p = preprocess_input(ia)
    feature = model_vgg16.predict(img_p)

    # Get image information
    path = os.path.dirname(img)
    file_name = os.path.basename(img)
    origin_address = path + '/' + file_name

    # Count image
    print('Reading Image:', file_name, '...')

    # Make predictions
    forecast = model_svm.predict(feature)

    # Image Discriminator:
    if forecast == 0:
        i = i+1
        print('farming')

    if forecast == 1:
        j = j+1
        print('forest_formation')

    if forecast == 2:
        k = k+1
        print('non_forest_area')

    if forecast == 3:
        m = m+1
        print('savanna_formation')

    if forecast == 4:
        n = n+1
        print('water')

sf.append(i)
sf.append(j)
sf.append(k)
sf.append(m)
sf.append(n)


# CHECKING WATER CLASS
wt = []   # water
i = 0     # counter
j = 0     # counter
k = 0     # counter
m = 0     # counter
n = 0     # counter

path = '../../data/dataset_cerradov3_NIR+G+B_splited_50k/test/water/*.tif'
for img in glob(os.path.join(os.getcwd(), path)):
   # Read the images
    ia = tf.keras.utils.load_img(img, target_size=(256, 256))

    # Transform them in array
    ia = tf.keras.utils.img_to_array(ia)
    ia = np.expand_dims(ia, axis=0)

    img_p = preprocess_input(ia)
    feature = model_vgg16.predict(img_p)

    # Get image information
    path = os.path.dirname(img)
    file_name = os.path.basename(img)
    origin_address = path + '/' + file_name

    # Count image
    print('Reading Image:', file_name, '...')

    # Make predictions
    forecast = model_svm.predict(feature)

    # Image Discriminator:
    if forecast == 0:
        i = i+1
        print('farming')

    if forecast == 1:
        j = j+1
        print('forest_formation')

    if forecast == 2:
        k = k+1
        print('non_forest_area')

    if forecast == 3:
        m = m+1
        print('savanna_formation')

    if forecast == 4:
        n = n+1
        print('water')

wt.append(i)
wt.append(j)
wt.append(k)
wt.append(m)
wt.append(n)



# ASSESSMENT OF MODEL
print('SVM Accuracy Assessmet Report:')

# Overall accuracy
sumAllClass = sum(fr) + sum(ff) + sum(nfa) + sum(sf) + sum(wt)
sumCorrectClassification = fr[0] + ff[1] + nfa[2] + sf[3] + wt[4]
overall_accuracy = round((sumCorrectClassification*100)/sumAllClass, 2)

# F1-Score
# tp = true positive
tp = sumCorrectClassification

# fp = false positive
fp_farming = ff[0] + nfa[0] + sf[0] + wt[0]
fp_forest_formation = fr[1] + nfa[1] + sf[1] + wt[1]
fp_non_forest_area = fr[2] + ff[2] + sf[2] + wt[2]
fp_savanna_formation = fr[3] + ff[3] + nfa[3] + wt[3]
fp_water = fr[4] + ff[4] + nfa[4] + sf[4]

fp = fp_farming + fp_forest_formation + fp_non_forest_area + fp_savanna_formation + fp_water

# fn = false negative
fn_farming = fr[1] + fr[2] + fr[3] + fr[4]
fn_forest_formation = ff[0] + ff[2] + ff[3] + ff[4]
fn_non_forest_area = nfa[0] + nfa[1] + nfa[3] + nfa[4]
fn_savanna_formation = sf[0] + sf[1] + sf[2] + sf[4]
fn_water = wt[0] + wt[1] + wt[2] + wt[3]
fn = fn_farming + fn_forest_formation + fn_non_forest_area + fn_savanna_formation + fn_water

# precision metric
precision = tp/(tp+fp)

# recall metric
recall = tp/(tp+fn)

# f1-score
f1Score = (2 * (precision * recall) / (precision + recall))

# Accuracy Farming
accuracyFarming = round(fr[0]*100/sum(fr),2)
print('1 Farming subset')
print('1.1 Accuracy: ', accuracyFarming)
print('1.2 Classifications: ')
print('- Farming: ', fr[0])
print('- Forest_formation: ', fr[1])
print('- Non_forest_area: ', fr[2])
print('- Savanna_formation: ', fr[3])
print('- Water: ', fr[4])
print('- Correct classification: ', fr[0])
print('- Incorrect classification: ', fn_farming)


# Accuracy Forest
accuracyForest = round(ff[1]*100/sum(ff),2)
print('2 Forest_formation subset')
print('2.1 Accuracy: ', accuracyForest)
print('2.2 Classifications: ')
print('- Farming: ', ff[0])
print('- Forest_formation: ', ff[1])
print('- Non_forest_area: ', ff[2])
print('- Savanna_formation: ', ff[3])
print('- Water: ', ff[4])
print('- Correct classification: ', ff[1])
print('- Incorrect classification: ', fn_forest_formation)


# Accuracy Non_Forest_Area
accuracyNonForestArea = round(nfa[2]*100/sum(nfa),2)
print('3 Non_Forest_Area subset')
print('3.1 Accuracy: ', accuracyNonForestArea)
print('3.2 Classifications: ')
print('- Farming: ', nfa[0])
print('- Forest_formation: ', nfa[1])
print('- Non_forest_area: ', nfa[2])
print('- Savanna_formation: ', nfa[3])
print('- Water: ', nfa[4])
print('- Correct classification: ', nfa[2])
print('- Incorrect classification: ', fn_non_forest_area)

# Accuracy Savanna_Formation
accuracySavannaFormation = round(sf[3]*100/sum(sf),2)
print('4 Savanna_Formation subset')
print('4.1 Accuracy: ', accuracySavannaFormation)
print('4.2 Classifications: ')
print('- Farming: ', sf[0])
print('- Forest_formation: ', sf[1])
print('- Non_forest_area: ', sf[2])
print('- Savanna_formation: ', sf[3])
print('- Water: ', sf[4])
print('- Correct classification: ', sf[3])
print('- Incorrect classification: ', fn_savanna_formation)

# Accuracy Water
accuracyWater = round(wt[4]*100/sum(wt),2)
print('5 Water subset')
print('5.1 Accuracy: ', accuracyWater)
print('5.2 Classifications: ')
print('- Farming: ', wt[0])
print('- Forest_formation: ', wt[1])
print('- Non_forest_area: ', wt[2])
print('- Savanna_formation: ', wt[3])
print('- Water: ', wt[4])
print('- Correct classification: ', wt[4])
print('- Incorrect classification: ', fn_water)

# Over all
print('6 Overall performance')
print('6.1 Accuracy: ', overall_accuracy)
print('6.2 Precision: ', precision)
print('6.3 Recall: ', recall)
print('6.4 F1-Score: ', f1Score)

Reading Image: 20200428_209_133_L4_155104.tif ...
farming
Reading Image: 20200428_209_133_L4_114158.tif ...
farming
Reading Image: 20200604_214_132_L4_13037.tif ...
farming
Reading Image: 20200609_213_128_L2_5128.tif ...
farming
Reading Image: 20200604_214_132_L4_13989.tif ...
farming
Reading Image: 20200614_212_131_L4_3638.tif ...
savanna_formation
Reading Image: 20200604_214_132_L4_10264.tif ...
farming
Reading Image: 20200804_208_120_L4_95128.tif ...
farming
Reading Image: 20200604_214_132_L4_9367.tif ...
farming
Reading Image: 20200428_209_133_L4_173128.tif ...
farming
Reading Image: 20200604_214_132_L4_113179.tif ...
farming
Reading Image: 20200428_209_133_L4_151148.tif ...
farming
Reading Image: 20200604_214_132_L4_120202.tif ...
farming
Reading Image: 20210505_209_135_L4_4785.tif ...
farming
Reading Image: 20200529_209_130_L4_18579.tif ...
farming
Reading Image: 20200428_209_133_L4_127182.tif ...
water
Reading Image: 20200428_209_133_L4_101179.tif ...
farming
Reading Image: 2020

Reading Image: 20200529_209_130_L4_18379.tif ...
farming
Reading Image: 20200529_209_130_L4_12668.tif ...
farming
Reading Image: 20200604_214_132_L4_10699.tif ...
farming
Reading Image: 20200428_209_132_L4_154170.tif ...
farming
Reading Image: 20200604_214_132_L4_9995.tif ...
farming
Reading Image: 20200609_213_128_L2_4436.tif ...
farming
Reading Image: 20200609_213_128_L2_6547.tif ...
farming
Reading Image: 20200604_214_131_L4_16237.tif ...
farming
Reading Image: 20200604_214_132_L4_11784.tif ...
farming
Reading Image: 20200609_213_128_L2_7843.tif ...
farming
Reading Image: 20200529_209_130_L4_126104.tif ...
farming
Reading Image: 20200428_209_133_L4_123136.tif ...
farming
Reading Image: 20200604_214_131_L4_13366.tif ...
farming
Reading Image: 20200428_209_133_L4_118177.tif ...
farming
Reading Image: 20200804_208_119_L4_9456.tif ...
farming
Reading Image: 20200609_213_128_L2_7658.tif ...
farming
Reading Image: 20200428_209_133_L4_96189.tif ...
farming
Reading Image: 20210901_210_136_L

Reading Image: 20200604_214_131_L4_23175.tif ...
farming
Reading Image: 20200609_213_128_L2_60117.tif ...
farming
Reading Image: 20200604_214_131_L4_11851.tif ...
farming
Reading Image: 20200428_209_133_L4_155199.tif ...
farming
Reading Image: 20200614_212_133_L4_85141.tif ...
water
Reading Image: 20200604_214_132_L4_117157.tif ...
farming
Reading Image: 20200604_214_131_L4_27105.tif ...
farming
Reading Image: 20200609_213_128_L2_3539.tif ...
farming
Reading Image: 20200804_208_119_L4_11829.tif ...
farming
Reading Image: 20200604_214_132_L4_104115.tif ...
farming
Reading Image: 20200529_209_130_L4_26178.tif ...
farming
Reading Image: 20200609_213_128_L2_8478.tif ...
farming
Reading Image: 20200604_214_132_L4_117155.tif ...
farming
Reading Image: 20200604_214_132_L4_8970.tif ...
farming
Reading Image: 20200609_213_128_L2_16281.tif ...
farming
Reading Image: 20200604_214_132_L4_94110.tif ...
farming
Reading Image: 20200604_214_131_L4_11853.tif ...
farming
Reading Image: 20200609_213_128_

Reading Image: 20200614_212_133_L4_83124.tif ...
forest_formation
Reading Image: 20200604_214_132_L4_10174.tif ...
farming
Reading Image: 20200428_209_133_L4_77114.tif ...
non_forest_area
Reading Image: 20200604_214_132_L4_123180.tif ...
farming
Reading Image: 20200604_214_132_L4_8396.tif ...
farming
Reading Image: 20210731_204_128_L4_81134.tif ...
farming
Reading Image: 20210901_210_134_L4_110150.tif ...
savanna_formation
Reading Image: 20200529_209_130_L4_13258.tif ...
farming
Reading Image: 20200804_208_119_L4_10660.tif ...
farming
Reading Image: 20200529_209_130_L4_131169.tif ...
water
Reading Image: 20200604_214_132_L4_12788.tif ...
farming
Reading Image: 20200604_214_132_L4_2491.tif ...
farming
Reading Image: 20200804_208_119_L4_73112.tif ...
farming
Reading Image: 20200428_209_132_L4_159193.tif ...
farming
Reading Image: 20200804_208_119_L4_9423.tif ...
farming
Reading Image: 20200609_213_128_L2_8278.tif ...
farming
Reading Image: 20200604_214_132_L4_11054.tif ...
farming
Readin

Reading Image: 20200604_214_132_L4_12778.tif ...
farming
Reading Image: 20200604_214_132_L4_118125.tif ...
farming
Reading Image: 20200804_208_119_L4_64111.tif ...
farming
Reading Image: 20200604_214_132_L4_116108.tif ...
farming
Reading Image: 20200604_214_132_L4_12750.tif ...
farming
Reading Image: 20200428_209_132_L4_149172.tif ...
farming
Reading Image: 20200604_214_131_L4_14881.tif ...
farming
Reading Image: 20210531_210_120_L4_154140.tif ...
water
Reading Image: 20200604_214_132_L4_108108.tif ...
farming
Reading Image: 20200604_214_131_L4_31142.tif ...
farming
Reading Image: 20200604_214_131_L4_14498.tif ...
farming
Reading Image: 20200604_214_131_L4_7998.tif ...
farming
Reading Image: 20200604_214_132_L4_121202.tif ...
farming
Reading Image: 20200604_214_131_L4_15744.tif ...
water
Reading Image: 20200604_214_131_L4_7999.tif ...
farming
Reading Image: 20200428_209_133_L4_123171.tif ...
farming
Reading Image: 20200428_209_132_L4_19798.tif ...
farming
Reading Image: 20210901_210_13

...

Reading Image: 20210922_212_131_L4_184190.tif ...
farming
Reading Image: 20210922_212_131_L4_106187.tif ...
water
Reading Image: 20210801_210_126_L4_14940.tif ...
water
Reading Image: 20210801_210_123_L4_14436.tif ...
water
Reading Image: 20210922_212_131_L4_18340.tif ...
water
Reading Image: 20210801_210_125_L4_5948.tif ...
water
Reading Image: 20200428_209_133_L4_4788.tif ...
water
Reading Image: 20210901_210_136_L4_8083.tif ...
non_forest_area
Reading Image: 20200614_212_131_L4_149207.tif ...
savanna_formation
Reading Image: 20200609_213_130_L4_14568.tif ...
water
Reading Image: 20200529_209_130_L4_35110.tif ...
water
Reading Image: 20210801_210_125_L4_5989.tif ...
water
Reading Image: 20200614_212_131_L4_11671.tif ...
savanna_formation
Reading Image: 20200604_214_131_L4_99170.tif ...
water
Reading Image: 20200604_214_131_L4_13275.tif ...
water
Reading Image: 20200604_214_131_L4_13261.tif ...
water
Reading Image: 20200614_212_131_L4_59147.tif ...
water
Reading Image: 20210706_209_12

Reading Image: 20200428_209_132_L4_147193.tif ...
farming
Reading Image: 20210801_210_126_L4_79148.tif ...
water
Reading Image: 20210526_211_131_L4_196109.tif ...
water
Reading Image: 20210801_210_133_L4_14951.tif ...
water
Reading Image: 20210806_209_121_L4_88167.tif ...
water
Reading Image: 20200604_214_131_L4_16082.tif ...
water
Reading Image: 20210424_205_136_L4_8498.tif ...
water
Reading Image: 20200604_214_131_L4_12582.tif ...
water
Reading Image: 20200604_214_131_L4_120117.tif ...
water
Reading Image: 20200614_212_132_L4_13685.tif ...
water
Reading Image: 20210726_205_121_L4_11467.tif ...
non_forest_area
Reading Image: 20200614_212_134_L4_62127.tif ...
water
Reading Image: 20200614_212_132_L4_150147.tif ...
water
Reading Image: 20210801_210_125_L4_6128.tif ...
water
Reading Image: 20200614_212_131_L4_13699.tif ...
savanna_formation
Reading Image: 20210801_210_123_L4_4173.tif ...
water
Reading Image: 20210917_213_129_L4_23178.tif ...
water
Reading Image: 20200614_212_133_L4_76191

Reading Image: 20200428_209_132_L4_136201.tif ...
non_forest_area
Reading Image: 20200804_208_120_L4_40185.tif ...
water
Reading Image: 20200804_208_120_L4_34162.tif ...
water
Reading Image: 20200614_212_134_L4_12183.tif ...
water
Reading Image: 20200604_214_131_L4_64180.tif ...
water
Reading Image: 20200614_212_132_L4_15953.tif ...
water
Reading Image: 20210811_208_125_L4_18059.tif ...
water
Reading Image: 20210701_210_121_L4_148153.tif ...
water
Reading Image: 20200604_214_131_L4_5618.tif ...
water
Reading Image: 20210721_206_132_L4_86167.tif ...
non_forest_area
Reading Image: 20210424_205_136_L4_165176.tif ...
water
Reading Image: 20200609_213_134_L4_10861.tif ...
water
Reading Image: 20200609_213_128_L2_4458.tif ...
water
Reading Image: 20200604_214_132_L4_112151.tif ...
water
Reading Image: 20200609_213_130_L4_26101.tif ...
water
Reading Image: 20200604_214_131_L4_6113.tif ...
water
Reading Image: 20200614_212_133_L4_47154.tif ...
water
Reading Image: 20200604_214_131_L4_14857.tif

Reading Image: 20210801_210_123_L4_92162.tif ...
water
Reading Image: 20210801_210_133_L4_38146.tif ...
water
Reading Image: 20200614_212_134_L4_22154.tif ...
water
Reading Image: 20200529_209_130_L4_96163.tif ...
water
Reading Image: 20200614_212_133_L4_113157.tif ...
water
Reading Image: 20210922_212_128_L4_106149.tif ...
water
Reading Image: 20200604_214_131_L4_6947.tif ...
water
Reading Image: 20210922_212_134_L4_16282.tif ...
water
Reading Image: 20210801_210_133_L4_14645.tif ...
water
Reading Image: 20200614_212_131_L4_17686.tif ...
savanna_formation
Reading Image: 20200614_212_134_L4_148194.tif ...
water
Reading Image: 20210625_205_123_L4_8430.tif ...
water
Reading Image: 20200609_213_128_L2_177153.tif ...
water
Reading Image: 20200604_214_132_L4_118145.tif ...
water
Reading Image: 20200614_212_131_L4_148206.tif ...
savanna_formation
Reading Image: 20200809_207_119_L4_179196.tif ...
forest_formation
Reading Image: 20210801_210_125_L4_15585.tif ...
water
Reading Image: 20210816_2

Reading Image: 20200609_213_128_L2_9697.tif ...
water
Reading Image: 20200604_214_131_L4_16571.tif ...
water
Reading Image: 20211007_209_129_L4_13049.tif ...
farming
Reading Image: 20210526_211_131_L4_136116.tif ...
water
Reading Image: 20210706_209_126_L4_7422.tif ...
water
Reading Image: 20200614_212_131_L4_187128.tif ...
water
Reading Image: 20200614_212_134_L4_164109.tif ...
water
Reading Image: 20200614_212_132_L4_3241.tif ...
water
Reading Image: 20200604_214_132_L4_112151 2.tif ...
water
Reading Image: 20200529_209_130_L4_30166.tif ...
water
Reading Image: 20211006_203_128_L4_13477.tif ...
water
Reading Image: 20200609_213_128_L2_145119.tif ...
water
Reading Image: 20200614_212_132_L4_150113.tif ...
water
Reading Image: 20200804_208_119_L4_11264.tif ...
water
Reading Image: 20200609_213_130_L4_6018.tif ...
water
Reading Image: 20200614_212_132_L4_12295.tif ...
water
Reading Image: 20200614_212_134_L4_94154.tif ...
water
Reading Image: 20210526_211_131_L4_12168.tif ...
savanna_fo

# RANDOM FOREST

In [22]:
rf = RandomForestClassifier(n_estimators=500).fit(d2_train_dataset, y_train)

In [23]:
yp_rf = rf.predict(d2_test_dataset)
accuracy_rf = accuracy_score(y_test, yp_rf)
f1score_rf = f1_score(y_test, yp_rf, average='micro')

In [24]:
print(accuracy_rf)
print(f1score_rf)

0.845
0.845


In [26]:

# CHECKING FARMING CLASS
fr = []   # farming
i = 0     # counter
j = 0     # counter
k = 0     # counter
m = 0     # counter
n = 0     # counter


path = '../../data/dataset_cerradov3_NIR+G+B_splited_50k/test/farming/*.tif'
for img in glob(os.path.join(os.getcwd(), path)):
    # Read the images
    ia = tf.keras.utils.load_img(img, target_size=(256, 256))

    # Transform them in array
    ia = tf.keras.utils.img_to_array(ia)
    ia = np.expand_dims(ia, axis=0)
    img_p = preprocess_input(ia)
    feature = model_vgg16.predict(img_p)

    # Get image information
    path = os.path.dirname(img)
    file_name = os.path.basename(img)
    origin_address = path + '/' + file_name

    # Count image
    print('Reading Image:', file_name, '...')

    # Make predictions
    forecast = rf.predict(feature)

    # Image Discriminator:
    if forecast == 0:
        i = i+1
        print('farming')

    if forecast == 1:
        j = j+1
        print('forest_formation')

    if forecast == 2:
        k = k+1
        print('non_forest_area')

    if forecast == 3:
        m = m+1
        print('savanna_formation')

    if forecast == 4:
        n = n+1
        print('water')

fr.append(i)
fr.append(j)
fr.append(k)
fr.append(m)
fr.append(n)

# CHECKING FOREST_FORMATION CLASS
ff = []   # forest_formation
i = 0     # counter
j = 0     # counter
k = 0     # counter
m = 0     # counter
n = 0     # counter

path = '../../data/dataset_cerradov3_NIR+G+B_splited_50k/test/forest_formation/*.tif'
for img in glob(os.path.join(os.getcwd(), path)):
    # Read the images
    ia = tf.keras.utils.load_img(img, target_size=(256, 256))

    # Transform them in array
    ia = tf.keras.utils.img_to_array(ia)
    ia = np.expand_dims(ia, axis=0)

    img_p = preprocess_input(ia)
    feature = model_vgg16.predict(img_p)

    # Get image information
    path = os.path.dirname(img)
    file_name = os.path.basename(img)
    origin_address = path + '/' + file_name

    # Count image
    print('Reading Image:', file_name, '...')

    # Make predictions
    forecast = rf.predict(feature)

    # Image Discriminator:
    if forecast == 0:
        i = i+1
        print('farming')

    if forecast == 1:
        j = j+1
        print('forest_formation')

    if forecast == 2:
        k = k+1
        print('non_forest_area')

    if forecast == 3:
        m = m+1
        print('savanna_formation')

    if forecast == 4:
        n = n+1
        print('water')

ff.append(i)
ff.append(j)
ff.append(k)
ff.append(m)
ff.append(n)

# CHECKING NON_FOREST_AREA CLASS
nfa = []  # non_forest_area
i = 0     # counter
j = 0     # counter
k = 0     # counter
m = 0     # counter
n = 0     # counter

path = '../../data/dataset_cerradov3_NIR+G+B_splited_50k/test/non_forest_area/*.tif'
for img in glob(os.path.join(os.getcwd(), path)):
    # Read the images
    ia = tf.keras.utils.load_img(img, target_size=(256, 256))

    # Transform them in array
    ia = tf.keras.utils.img_to_array(ia)
    ia = np.expand_dims(ia, axis=0)

    img_p = preprocess_input(ia)
    feature = model_vgg16.predict(img_p)

    # Get image information
    path = os.path.dirname(img)
    file_name = os.path.basename(img)
    origin_address = path + '/' + file_name

    # Count image
    print('Reading Image:', file_name, '...')

    # Make predictions
    forecast = rf.predict(feature)

    # Image Discriminator:
    if forecast == 0:
        i = i+1
        print('farming')

    if forecast == 1:
        j = j+1
        print('forest_formation')

    if forecast == 2:
        k = k+1
        print('non_forest_area')

    if forecast == 3:
        m = m+1
        print('savanna_formation')

    if forecast == 4:
        n = n+1
        print('water')

nfa.append(i)
nfa.append(j)
nfa.append(k)
nfa.append(m)
nfa.append(n)


# CHECKING SAVANNA_FORMATION CLASS
sf = []   # savanna_formation
i = 0     # counter
j = 0     # counter
k = 0     # counter
m = 0     # counter
n = 0     # counter

path = '../../data/dataset_cerradov3_NIR+G+B_splited_50k/test/savanna_formation/*.tif'
for img in glob(os.path.join(os.getcwd(), path)):
    # Read the images
    ia = tf.keras.utils.load_img(img, target_size=(256, 256))

    # Transform them in array
    ia = tf.keras.utils.img_to_array(ia)
    ia = np.expand_dims(ia, axis=0)

    img_p = preprocess_input(ia)
    feature = model_vgg16.predict(img_p)

    # Get image information
    path = os.path.dirname(img)
    file_name = os.path.basename(img)
    origin_address = path + '/' + file_name

    # Count image
    print('Reading Image:', file_name, '...')

    # Make predictions
    forecast = rf.predict(feature)

    # Image Discriminator:
    if forecast == 0:
        i = i+1
        print('farming')

    if forecast == 1:
        j = j+1
        print('forest_formation')

    if forecast == 2:
        k = k+1
        print('non_forest_area')

    if forecast == 3:
        m = m+1
        print('savanna_formation')

    if forecast == 4:
        n = n+1
        print('water')

sf.append(i)
sf.append(j)
sf.append(k)
sf.append(m)
sf.append(n)


# CHECKING WATER CLASS
wt = []   # water
i = 0     # counter
j = 0     # counter
k = 0     # counter
m = 0     # counter
n = 0     # counter

path = '../../data/dataset_cerradov3_NIR+G+B_splited_50k/test/water/*.tif'
for img in glob(os.path.join(os.getcwd(), path)):
   # Read the images
    ia = tf.keras.utils.load_img(img, target_size=(256, 256))

    # Transform them in array
    ia = tf.keras.utils.img_to_array(ia)
    ia = np.expand_dims(ia, axis=0)

    img_p = preprocess_input(ia)
    feature = model_vgg16.predict(img_p)

    # Get image information
    path = os.path.dirname(img)
    file_name = os.path.basename(img)
    origin_address = path + '/' + file_name

    # Count image
    print('Reading Image:', file_name, '...')

    # Make predictions
    forecast = rf.predict(feature)

    # Image Discriminator:
    if forecast == 0:
        i = i+1
        print('farming')

    if forecast == 1:
        j = j+1
        print('forest_formation')

    if forecast == 2:
        k = k+1
        print('non_forest_area')

    if forecast == 3:
        m = m+1
        print('savanna_formation')

    if forecast == 4:
        n = n+1
        print('water')

wt.append(i)
wt.append(j)
wt.append(k)
wt.append(m)
wt.append(n)



# ASSESSMENT OF MODEL
print('RF Accuracy Assessmet Report:')

# Overall accuracy
sumAllClass = sum(fr) + sum(ff) + sum(nfa) + sum(sf) + sum(wt)
sumCorrectClassification = fr[0] + ff[1] + nfa[2] + sf[3] + wt[4]
overall_accuracy = round((sumCorrectClassification*100)/sumAllClass, 2)

# F1-Score
# tp = true positive
tp = sumCorrectClassification

# fp = false positive
fp_farming = ff[0] + nfa[0] + sf[0] + wt[0]
fp_forest_formation = fr[1] + nfa[1] + sf[1] + wt[1]
fp_non_forest_area = fr[2] + ff[2] + sf[2] + wt[2]
fp_savanna_formation = fr[3] + ff[3] + nfa[3] + wt[3]
fp_water = fr[4] + ff[4] + nfa[4] + sf[4]

fp = fp_farming + fp_forest_formation + fp_non_forest_area + fp_savanna_formation + fp_water

# fn = false negative
fn_farming = fr[1] + fr[2] + fr[3] + fr[4]
fn_forest_formation = ff[0] + ff[2] + ff[3] + ff[4]
fn_non_forest_area = nfa[0] + nfa[1] + nfa[3] + nfa[4]
fn_savanna_formation = sf[0] + sf[1] + sf[2] + sf[4]
fn_water = wt[0] + wt[1] + wt[2] + wt[3]
fn = fn_farming + fn_forest_formation + fn_non_forest_area + fn_savanna_formation + fn_water

# precision metric
precision = tp/(tp+fp)

# recall metric
recall = tp/(tp+fn)

# f1-score
f1Score = (2 * (precision * recall) / (precision + recall))

# Accuracy Farming
accuracyFarming = round(fr[0]*100/sum(fr),2)
print('1 Farming subset')
print('1.1 Accuracy: ', accuracyFarming)
print('1.2 Classifications: ')
print('- Farming: ', fr[0])
print('- Forest_formation: ', fr[1])
print('- Non_forest_area: ', fr[2])
print('- Savanna_formation: ', fr[3])
print('- Water: ', fr[4])
print('- Correct classification: ', fr[0])
print('- Incorrect classification: ', fn_farming)


# Accuracy Forest
accuracyForest = round(ff[1]*100/sum(ff),2)
print('2 Forest_formation subset')
print('2.1 Accuracy: ', accuracyForest)
print('2.2 Classifications: ')
print('- Farming: ', ff[0])
print('- Forest_formation: ', ff[1])
print('- Non_forest_area: ', ff[2])
print('- Savanna_formation: ', ff[3])
print('- Water: ', ff[4])
print('- Correct classification: ', ff[1])
print('- Incorrect classification: ', fn_forest_formation)


# Accuracy Non_Forest_Area
accuracyNonForestArea = round(nfa[2]*100/sum(nfa),2)
print('3 Non_Forest_Area subset')
print('3.1 Accuracy: ', accuracyNonForestArea)
print('3.2 Classifications: ')
print('- Farming: ', nfa[0])
print('- Forest_formation: ', nfa[1])
print('- Non_forest_area: ', nfa[2])
print('- Savanna_formation: ', nfa[3])
print('- Water: ', nfa[4])
print('- Correct classification: ', nfa[2])
print('- Incorrect classification: ', fn_non_forest_area)

# Accuracy Savanna_Formation
accuracySavannaFormation = round(sf[3]*100/sum(sf),2)
print('4 Savanna_Formation subset')
print('4.1 Accuracy: ', accuracySavannaFormation)
print('4.2 Classifications: ')
print('- Farming: ', sf[0])
print('- Forest_formation: ', sf[1])
print('- Non_forest_area: ', sf[2])
print('- Savanna_formation: ', sf[3])
print('- Water: ', sf[4])
print('- Correct classification: ', sf[3])
print('- Incorrect classification: ', fn_savanna_formation)

# Accuracy Water
accuracyWater = round(wt[4]*100/sum(wt),2)
print('5 Water subset')
print('5.1 Accuracy: ', accuracyWater)
print('5.2 Classifications: ')
print('- Farming: ', wt[0])
print('- Forest_formation: ', wt[1])
print('- Non_forest_area: ', wt[2])
print('- Savanna_formation: ', wt[3])
print('- Water: ', wt[4])
print('- Correct classification: ', wt[4])
print('- Incorrect classification: ', fn_water)

# Over all
print('6 Overall performance')
print('6.1 Accuracy: ', overall_accuracy)
print('6.2 Precision: ', precision)
print('6.3 Recall: ', recall)
print('6.4 F1-Score: ', f1Score)

Reading Image: 20200428_209_133_L4_155104.tif ...
farming
Reading Image: 20200428_209_133_L4_114158.tif ...
farming
Reading Image: 20200604_214_132_L4_13037.tif ...
farming
Reading Image: 20200609_213_128_L2_5128.tif ...
farming
Reading Image: 20200604_214_132_L4_13989.tif ...
farming
Reading Image: 20200614_212_131_L4_3638.tif ...
savanna_formation
Reading Image: 20200604_214_132_L4_10264.tif ...
farming
Reading Image: 20200804_208_120_L4_95128.tif ...
farming
Reading Image: 20200604_214_132_L4_9367.tif ...
farming
Reading Image: 20200428_209_133_L4_173128.tif ...
farming
Reading Image: 20200604_214_132_L4_113179.tif ...
farming
Reading Image: 20200428_209_133_L4_151148.tif ...
farming
Reading Image: 20200604_214_132_L4_120202.tif ...
farming
Reading Image: 20210505_209_135_L4_4785.tif ...
farming
Reading Image: 20200529_209_130_L4_18579.tif ...
farming
Reading Image: 20200428_209_133_L4_127182.tif ...
water
Reading Image: 20200428_209_133_L4_101179.tif ...
farming
Reading Image: 2020

Reading Image: 20200529_209_130_L4_18379.tif ...
farming
Reading Image: 20200529_209_130_L4_12668.tif ...
farming
Reading Image: 20200604_214_132_L4_10699.tif ...
farming
Reading Image: 20200428_209_132_L4_154170.tif ...
farming
Reading Image: 20200604_214_132_L4_9995.tif ...
farming
Reading Image: 20200609_213_128_L2_4436.tif ...
farming
Reading Image: 20200609_213_128_L2_6547.tif ...
farming
Reading Image: 20200604_214_131_L4_16237.tif ...
farming
Reading Image: 20200604_214_132_L4_11784.tif ...
farming
Reading Image: 20200609_213_128_L2_7843.tif ...
farming
Reading Image: 20200529_209_130_L4_126104.tif ...
farming
Reading Image: 20200428_209_133_L4_123136.tif ...
farming
Reading Image: 20200604_214_131_L4_13366.tif ...
farming
Reading Image: 20200428_209_133_L4_118177.tif ...
farming
Reading Image: 20200804_208_119_L4_9456.tif ...
farming
Reading Image: 20200609_213_128_L2_7658.tif ...
farming
Reading Image: 20200428_209_133_L4_96189.tif ...
farming
Reading Image: 20210901_210_136_L

Reading Image: 20200609_213_128_L2_5412.tif ...
farming
Reading Image: 20200604_214_131_L4_23175.tif ...
farming
Reading Image: 20200609_213_128_L2_60117.tif ...
farming
Reading Image: 20200604_214_131_L4_11851.tif ...
farming
Reading Image: 20200428_209_133_L4_155199.tif ...
farming
Reading Image: 20200614_212_133_L4_85141.tif ...
water
Reading Image: 20200604_214_132_L4_117157.tif ...
farming
Reading Image: 20200604_214_131_L4_27105.tif ...
farming
Reading Image: 20200609_213_128_L2_3539.tif ...
farming
Reading Image: 20200804_208_119_L4_11829.tif ...
farming
Reading Image: 20200604_214_132_L4_104115.tif ...
farming
Reading Image: 20200529_209_130_L4_26178.tif ...
farming
Reading Image: 20200609_213_128_L2_8478.tif ...
farming
Reading Image: 20200604_214_132_L4_117155.tif ...
farming
Reading Image: 20200604_214_132_L4_8970.tif ...
farming
Reading Image: 20200609_213_128_L2_16281.tif ...
farming
Reading Image: 20200604_214_132_L4_94110.tif ...
farming
Reading Image: 20200604_214_131_L

Reading Image: 20200604_214_132_L4_15193.tif ...
farming
Reading Image: 20200604_214_132_L4_13290.tif ...
farming
Reading Image: 20200614_212_133_L4_83124.tif ...
forest_formation
Reading Image: 20200604_214_132_L4_10174.tif ...
farming
Reading Image: 20200428_209_133_L4_77114.tif ...
non_forest_area
Reading Image: 20200604_214_132_L4_123180.tif ...
farming
Reading Image: 20200604_214_132_L4_8396.tif ...
farming
Reading Image: 20210731_204_128_L4_81134.tif ...
farming
Reading Image: 20210901_210_134_L4_110150.tif ...
savanna_formation
Reading Image: 20200529_209_130_L4_13258.tif ...
farming
Reading Image: 20200804_208_119_L4_10660.tif ...
farming
Reading Image: 20200529_209_130_L4_131169.tif ...
water
Reading Image: 20200604_214_132_L4_12788.tif ...
farming
Reading Image: 20200604_214_132_L4_2491.tif ...
farming
Reading Image: 20200804_208_119_L4_73112.tif ...
farming
Reading Image: 20200428_209_132_L4_159193.tif ...
farming
Reading Image: 20200804_208_119_L4_9423.tif ...
farming
Readi

water
Reading Image: 20200804_208_119_L4_83167.tif ...
farming
Reading Image: 20200604_214_132_L4_121149.tif ...
farming
Reading Image: 20200609_213_128_L2_7597.tif ...
farming
Reading Image: 20200604_214_132_L4_12778.tif ...
farming
Reading Image: 20200604_214_132_L4_118125.tif ...
farming
Reading Image: 20200804_208_119_L4_64111.tif ...
farming
Reading Image: 20200604_214_132_L4_116108.tif ...
farming
Reading Image: 20200604_214_132_L4_12750.tif ...
farming
Reading Image: 20200428_209_132_L4_149172.tif ...
farming
Reading Image: 20200604_214_131_L4_14881.tif ...
farming
Reading Image: 20210531_210_120_L4_154140.tif ...
water
Reading Image: 20200604_214_132_L4_108108.tif ...
farming
Reading Image: 20200604_214_131_L4_31142.tif ...
farming
Reading Image: 20200604_214_131_L4_14498.tif ...
farming
Reading Image: 20200604_214_131_L4_7998.tif ...
farming
Reading Image: 20200604_214_132_L4_121202.tif ...
farming
Reading Image: 20200604_214_131_L4_15744.tif ...
water
Reading Image: 20200604_

...

Reading Image: 20210922_212_131_L4_184190.tif ...
farming
Reading Image: 20210922_212_131_L4_106187.tif ...
water
Reading Image: 20210801_210_126_L4_14940.tif ...
water
Reading Image: 20210801_210_123_L4_14436.tif ...
water
Reading Image: 20210922_212_131_L4_18340.tif ...
water
Reading Image: 20210801_210_125_L4_5948.tif ...
water
Reading Image: 20200428_209_133_L4_4788.tif ...
water
Reading Image: 20210901_210_136_L4_8083.tif ...
non_forest_area
Reading Image: 20200614_212_131_L4_149207.tif ...
savanna_formation
Reading Image: 20200609_213_130_L4_14568.tif ...
water
Reading Image: 20200529_209_130_L4_35110.tif ...
water
Reading Image: 20210801_210_125_L4_5989.tif ...
water
Reading Image: 20200614_212_131_L4_11671.tif ...
water
Reading Image: 20200604_214_131_L4_99170.tif ...
water
Reading Image: 20200604_214_131_L4_13275.tif ...
water
Reading Image: 20200604_214_131_L4_13261.tif ...
water
Reading Image: 20200614_212_131_L4_59147.tif ...
water
Reading Image: 20210706_209_126_L4_6418.ti

Reading Image: 20210801_210_126_L4_79148.tif ...
water
Reading Image: 20210526_211_131_L4_196109.tif ...
water
Reading Image: 20210801_210_133_L4_14951.tif ...
farming
Reading Image: 20210806_209_121_L4_88167.tif ...
water
Reading Image: 20200604_214_131_L4_16082.tif ...
water
Reading Image: 20210424_205_136_L4_8498.tif ...
water
Reading Image: 20200604_214_131_L4_12582.tif ...
water
Reading Image: 20200604_214_131_L4_120117.tif ...
water
Reading Image: 20200614_212_132_L4_13685.tif ...
water
Reading Image: 20210726_205_121_L4_11467.tif ...
water
Reading Image: 20200614_212_134_L4_62127.tif ...
water
Reading Image: 20200614_212_132_L4_150147.tif ...
water
Reading Image: 20210801_210_125_L4_6128.tif ...
water
Reading Image: 20200614_212_131_L4_13699.tif ...
savanna_formation
Reading Image: 20210801_210_123_L4_4173.tif ...
water
Reading Image: 20210917_213_129_L4_23178.tif ...
water
Reading Image: 20200614_212_133_L4_76191.tif ...
water
Reading Image: 20200809_207_119_L4_168209.tif ...
w

Reading Image: 20200804_208_120_L4_40185.tif ...
water
Reading Image: 20200804_208_120_L4_34162.tif ...
water
Reading Image: 20200614_212_134_L4_12183.tif ...
water
Reading Image: 20200604_214_131_L4_64180.tif ...
water
Reading Image: 20200614_212_132_L4_15953.tif ...
water
Reading Image: 20210811_208_125_L4_18059.tif ...
water
Reading Image: 20210701_210_121_L4_148153.tif ...
water
Reading Image: 20200604_214_131_L4_5618.tif ...
water
Reading Image: 20210721_206_132_L4_86167.tif ...
non_forest_area
Reading Image: 20210424_205_136_L4_165176.tif ...
savanna_formation
Reading Image: 20200609_213_134_L4_10861.tif ...
farming
Reading Image: 20200609_213_128_L2_4458.tif ...
water
Reading Image: 20200604_214_132_L4_112151.tif ...
water
Reading Image: 20200609_213_130_L4_26101.tif ...
non_forest_area
Reading Image: 20200604_214_131_L4_6113.tif ...
water
Reading Image: 20200614_212_133_L4_47154.tif ...
water
Reading Image: 20200604_214_131_L4_14857.tif ...
water
Reading Image: 20210901_210_134

farming
Reading Image: 20200614_212_131_L4_146161.tif ...
water
Reading Image: 20210801_210_123_L4_92162.tif ...
non_forest_area
Reading Image: 20210801_210_133_L4_38146.tif ...
water
Reading Image: 20200614_212_134_L4_22154.tif ...
water
Reading Image: 20200529_209_130_L4_96163.tif ...
water
Reading Image: 20200614_212_133_L4_113157.tif ...
water
Reading Image: 20210922_212_128_L4_106149.tif ...
water
Reading Image: 20200604_214_131_L4_6947.tif ...
water
Reading Image: 20210922_212_134_L4_16282.tif ...
water
Reading Image: 20210801_210_133_L4_14645.tif ...
water
Reading Image: 20200614_212_131_L4_17686.tif ...
savanna_formation
Reading Image: 20200614_212_134_L4_148194.tif ...
savanna_formation
Reading Image: 20210625_205_123_L4_8430.tif ...
water
Reading Image: 20200609_213_128_L2_177153.tif ...
water
Reading Image: 20200604_214_132_L4_118145.tif ...
water
Reading Image: 20200614_212_131_L4_148206.tif ...
savanna_formation
Reading Image: 20200809_207_119_L4_179196.tif ...
water
Readi

Reading Image: 20200609_213_130_L4_15787.tif ...
water
Reading Image: 20200609_213_128_L2_9697.tif ...
water
Reading Image: 20200604_214_131_L4_16571.tif ...
water
Reading Image: 20211007_209_129_L4_13049.tif ...
farming
Reading Image: 20210526_211_131_L4_136116.tif ...
water
Reading Image: 20210706_209_126_L4_7422.tif ...
water
Reading Image: 20200614_212_131_L4_187128.tif ...
savanna_formation
Reading Image: 20200614_212_134_L4_164109.tif ...
water
Reading Image: 20200614_212_132_L4_3241.tif ...
water
Reading Image: 20200604_214_132_L4_112151 2.tif ...
water
Reading Image: 20200529_209_130_L4_30166.tif ...
water
Reading Image: 20211006_203_128_L4_13477.tif ...
water
Reading Image: 20200609_213_128_L2_145119.tif ...
water
Reading Image: 20200614_212_132_L4_150113.tif ...
water
Reading Image: 20200804_208_119_L4_11264.tif ...
water
Reading Image: 20200609_213_130_L4_6018.tif ...
water
Reading Image: 20200614_212_132_L4_12295.tif ...
water
Reading Image: 20200614_212_134_L4_94154.tif ..